# Universidade Federal de Minas Gerais
## Computação Evolucionária - TTC
### Trabalho Prático 4

Daniela Amaral Sampaio - 2017074351

Matheus Brito Faria - 2017074386

## 1. Introdução

O trabalho prático 4 tem como objetivo implementar o algoritmo Particle Swarm Optimization (PSO) tomando como base o pseudocódigo fornecido pelo professor e testar os seguintes problemas multimodais de otimização contínua:

```
a.   peaks, para -3 <= x1 <= 3, -3 <= x2 <= 3 com N = 50; mínimo global em
x*=[0.228,-1.625] com f(x*) = -6.5511;

b.   rastrigin, para -2 <= x1 <= 2, -2<= x2 <= 2 com N = 50; mínimo global em
x*=[0,0] com f(x*) = -20.
```

O PSO é um algoritmo meta-heurístico baseado no comportamento social de um bando de pássaros. O método tem como objetivo buscar a solução ótima, em um espaço de busca, através da troca de informações entre indivíduos de uma população determinando qual trajetória cada um deles deverá tomar no espaço de busca.

In [ ]:
import numpy as np
from typing import Callable

## 2. Implementação
Para realizar a implementação foi implementada a classe ```class PSOlver```.

### 2.1. Classe Particle Swarm Optimization (PSO)

Inicia-se criando as entradas necessárias, como a dimensão do problema, etc. Após isso, ceta as constantes como o valor máximo de iterações, tamanho do swarm, número de variáveis, velocidade máxima. Após isso, inicia-se a população e começa a atualizar os valores de velocidade de cada partícula. Para cada partícula, irá avaliar a aptidão. Se o valor da aptidão do indivíduo analisado for melhor que a melhor aptidão até o momento, então atualiza esse valor. Então, obtem qual a melhor posição global avaliando qual o mínimo entre os p vizinhos. Para cada dimensão d, atualiza a componente de velocidade. Se essa componente de velocidade ultrapassar o valor da velocidade máxima, então irá refletir a partícula no espaço de busca. Após ter esse vetor de velocidade calculado, desloca as partículas para suas novas posições. O processo acaba quando se finalizam o máximo de iterações indicados. Por fim, são printados na tela os valores de mínimo local e posição em cada uma das suas iterações. 

In [ ]:
class PSOlver:
    def __init__(self,
                 loss_function: Callable,
                 lower_bound: int,
                 upper_bound: int,) -> None:

        self.loss_function = loss_function
        self.LOWER_BOUND = lower_bound
        self.UPPER_BOUND = upper_bound
        self.random_generator = np.random.default_rng()

    def _set_constants(self, parameters: dict) -> None:
        self.MAX_ITERATIONS = parameters.get("max_iterations")
        self.SWARM_SIZE = parameters.get("swarm_size")
        self.NUMBER_VARIABLES = parameters.get("number_variables")
        self.MAX_VELOCITY = parameters.get("max_velocity")
        self.INITIAL_INERTIA_WEIGHT = parameters.get(
            "initial_inertia_weight")
        self.ACCELERATION_COGNITIVE = parameters.get(
            "acceleration_cognitive")
        self.ACCELERATION_SOCIAL = parameters.get("acceleration_social")

    def _to_initilize(self) -> None:
        self.positions = self.random_generator.uniform(
            low=self.LOWER_BOUND, high=self.UPPER_BOUND, size=(
                self.SWARM_SIZE, self.NUMBER_VARIABLES)
        )

        self.velocities = self.random_generator.uniform(
            low=-self.MAX_VELOCITY, high=self.MAX_VELOCITY, size=(
                self.SWARM_SIZE, self.NUMBER_VARIABLES)
        )

    def _to_evaluete(self) -> None:
        self.loss = np.array(list(map(self.loss_function, self.positions)))

    def _get_best_position(self, first_call: bool = False) -> None:
        arg_best_loss = np.argmin(self.loss)
        if first_call:
            self.best_global_loss = self.loss[arg_best_loss]
            self.best_global_position = self.positions[arg_best_loss]
            self.best_individual_loss = self.loss
            self.best_individual_position = self.positions
        else:
            if self.loss[arg_best_loss] < self.best_global_loss:
                self.best_global_loss = self.loss[arg_best_loss]
                self.best_global_position = self.positions[arg_best_loss]

            arg_best_loss_individual = self.loss < self.best_individual_loss
            self.best_individual_loss[arg_best_loss_individual] = (
                self.loss[arg_best_loss_individual]
            )
            self.best_individual_position[arg_best_loss_individual] = (
                self.positions[arg_best_loss_individual]
            )

    def _to_update_velocities(self) -> None:
        random_variables = self.random_generator.uniform(size=2)

        self.velocities = (self.inertia_weight * self.velocities) + \
            (random_variables[0]*self.ACCELERATION_COGNITIVE *
             (self.best_individual_position-self.positions)) + \
            (random_variables[1]*self.ACCELERATION_SOCIAL *
             (self.best_global_position-self.positions))

        trespassing_max_speed = np.abs(self.velocities) > self.MAX_VELOCITY

        while trespassing_max_speed.any():
            self.velocities[trespassing_max_speed] = (
                np.sign(self.velocities[trespassing_max_speed])*(
                    np.abs(self.velocities[trespassing_max_speed]) -
                    self.MAX_VELOCITY)
            )
            trespassing_max_speed = np.abs(
                self.velocities) > self.MAX_VELOCITY

    def _to_update_positions(self) -> None:
        self.positions = self.positions + self.velocities

    def _to_update_inertia_weight(self, iteration) -> None:
        self.inertia_weight = self.INITIAL_INERTIA_WEIGHT - \
            (((self.INITIAL_INERTIA_WEIGHT)/self.MAX_ITERATIONS)*iteration)

    def run(self,
            max_iterations: int,
            swarm_size: int,
            number_variables: int,
            max_velocity: float,
            initial_inertia_weight: float,
            acceleration_cognitive: float,
            acceleration_social: float,
            ) -> None:

        self._set_constants(locals())
        self._to_initilize()
        self._to_evaluete()
        self._get_best_position(first_call=True)

        for iteration in range(self.MAX_ITERATIONS):
            print(
                f"[ITERATION {str(iteration).zfill(2)}] "
                f"f(x*)={self.best_global_loss}, "
                f"x*={self.best_global_position}")

            self._to_update_inertia_weight(iteration)
            self._to_update_velocities()
            self._to_update_positions()
            self._to_evaluete()
            self._get_best_position()

### 2.2. Função Objetivo *peaks*

In [ ]:
def peaks(x):
    x = x.T
    F = (
        3 * (1 - x[0]) ** 2 * np.exp(-(x[0] ** 2) - (x[1] + 1) ** 2)
        - 10 * (x[0] / 5 - x[0] ** 3 - x[1] ** 5) *
        np.exp(-x[0] ** 2 - x[1] ** 2)
        - 1 / 3 * np.exp(-((x[0] + 1) ** 2) - x[1] ** 2)
    )
    return F

### 2.3. Função Objetivo *rastrigin*

In [ ]:
def rastrigin(x):
    x = x.reshape(1, -1).T
    Q = np.eye(len(x))
    X = Q.dot(x)

    n = len(X)
    F = 0

    for i in range(n):
        F = F + X[i]**2 - 10*np.cos(2*np.pi*X[i])

    return F[0]

## 3. Resultados

Nesse tópico, faremos o teste do algoritmo implementado para cada uma das duas funções objetivos com suas determinadas variáveis de entrada e analisaremos se os resultados foram satisfatórios.

### 3.1. Resultados da Função Objetivo *peaks*

In [ ]:
solver_peaks = PSOlver(peaks, lower_bound=-3, upper_bound=3)

solver_peaks.run(max_iterations=30,
                 swarm_size=50,
                 number_variables=2,
                 max_velocity=1.5,
                 initial_inertia_weight=1,
                 acceleration_cognitive=0.8,
                 acceleration_social=0.5)

[ITERATION 00] f(x*)=-6.246178841425272, x*=[ 0.32889094 -1.49188856]
[ITERATION 01] f(x*)=-6.246178841425272, x*=[ 0.32889094 -1.49188856]
[ITERATION 02] f(x*)=-6.246178841425272, x*=[ 0.32889094 -1.49188856]
[ITERATION 03] f(x*)=-6.246178841425272, x*=[ 0.32889094 -1.49188856]
[ITERATION 04] f(x*)=-6.246178841425272, x*=[ 0.32889094 -1.49188856]
[ITERATION 05] f(x*)=-6.442102531351385, x*=[ 0.309098   -1.55772501]
[ITERATION 06] f(x*)=-6.442102531351385, x*=[ 0.309098   -1.55772501]
[ITERATION 07] f(x*)=-6.442102531351385, x*=[ 0.309098   -1.55772501]
[ITERATION 08] f(x*)=-6.451061748439946, x*=[ 0.24229075 -1.54183427]
[ITERATION 09] f(x*)=-6.531045653062599, x*=[ 0.26008501 -1.64890695]
[ITERATION 10] f(x*)=-6.531045653062599, x*=[ 0.26008501 -1.64890695]
[ITERATION 11] f(x*)=-6.531045653062599, x*=[ 0.26008501 -1.64890695]
[ITERATION 12] f(x*)=-6.548800660072332, x*=[ 0.24254936 -1.62914461]
[ITERATION 13] f(x*)=-6.548800660072332, x*=[ 0.24254936 -1.62914461]
[ITERATION 14] f(x*)

Visto que o objetivo seria o mínimo global em x*=[0.228 , -1.625] com f(x*) = -6.5511, o teste o algoritmo atendeu com sucesso, visto que o resultado na iteração 29 foi de x*=[0.22826599 , -1.6255379] e f(x*)= -6.551133331247272.

### 3.2. Resultados da Função Objetivo *rastrigin*

In [ ]:
solver_rastrigin = PSOlver(rastrigin, -2, 2)

solver_rastrigin.run(max_iterations=30,
                     swarm_size=50,
                     number_variables=2,
                     max_velocity=1.5,
                     initial_inertia_weight=0.6,
                     acceleration_cognitive=1.2,
                     acceleration_social=0.7)

[ITERATION 00] f(x*)=-16.48136697495618, x*=[-0.10605598  1.0384096 ]
[ITERATION 01] f(x*)=-16.48136697495618, x*=[-0.10605598  1.0384096 ]
[ITERATION 02] f(x*)=-19.04123565203811, x*=[-0.01165846  0.06907136]
[ITERATION 03] f(x*)=-19.04123565203811, x*=[-0.01165846  0.06907136]
[ITERATION 04] f(x*)=-19.04123565203811, x*=[-0.01165846  0.06907136]
[ITERATION 05] f(x*)=-19.99704318359767, x*=[-0.00124221 -0.00365533]
[ITERATION 06] f(x*)=-19.99704318359767, x*=[-0.00124221 -0.00365533]
[ITERATION 07] f(x*)=-19.99704318359767, x*=[-0.00124221 -0.00365533]
[ITERATION 08] f(x*)=-19.99704318359767, x*=[-0.00124221 -0.00365533]
[ITERATION 09] f(x*)=-19.99704318359767, x*=[-0.00124221 -0.00365533]
[ITERATION 10] f(x*)=-19.99704318359767, x*=[-0.00124221 -0.00365533]
[ITERATION 11] f(x*)=-19.99704318359767, x*=[-0.00124221 -0.00365533]
[ITERATION 12] f(x*)=-19.99704318359767, x*=[-0.00124221 -0.00365533]
[ITERATION 13] f(x*)=-19.99704318359767, x*=[-0.00124221 -0.00365533]
[ITERATION 14] f(x*)

Visto que o objetivo seria o mínimo global em x*=[0 , 0] com f(x*) = -20, o teste o algoritmo atendeu com sucesso, visto que o resultado na iteração 29 foi de x*=[8.94928425e-07 , 4.98784028e-07] e f(x*)= -19.9999999997917, que é muito próximo do objetivo.

## 4. Conclusão

Durante a execução do trabalho foi possível utilizar na prática conceitos aprendidos em sala de aula e acredita-se que o resultado final do trabalho tenha sido satisfatório, visto que o algoritmo Particle Swarm Optimization (PSO) foi implementado com sucesso e nos dois testes de *peaks* e *rastrigin* foram obtidos resultados muito próximos ou iguais aos esperados de mínimos global.